In [ ]:
%pip install psycopg2-binary sqlalchemy pandas matplotlib dotenv

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (12, 5)
plt.rcParams["axes.grid"] = True

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 140)

In [ ]:
from sqlalchemy import create_engine
from dotenv import load_dotenv

load_dotenv()

DB_HOST = os.getenv("DB_HOST", "postgres")
DB_PORT = os.getenv("DB_PORT", "5432")
DB_NAME = os.getenv("DB_NAME", "fit")
DB_USER = os.getenv("DB_USER", "dan")
DB_PASS = os.getenv("DB_PASSWORD", "dan")

dsn = f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(dsn)

with engine.connect() as conn:
    print("DB OK:", conn.exec_driver_sql("select now()").scalar())

In [ ]:
q_last = """
SELECT
  id,
  created_at,
  activity_type,
  steps,
  calories
FROM activity
ORDER BY created_at DESC
LIMIT 50;
"""

df_last = pd.read_sql(q_last, engine)
df_last

In [ ]:
q_top = """
SELECT
  activity_type,
  COUNT(*)      AS records,
  SUM(steps)    AS steps_sum,
  SUM(calories) AS calories_sum
FROM activity
GROUP BY 1
ORDER BY steps_sum DESC;
"""

df_top = pd.read_sql(q_top, engine)
df_top

In [ ]:
df_plot = df_top.sort_values("steps_sum", ascending=True)

plt.figure()
plt.barh(df_plot["activity_type"], df_plot["steps_sum"])
plt.title("Топ типов активности (сумма шагов)")
plt.xlabel("steps_sum")
plt.ylabel("activity_type")
plt.show()

In [ ]:
q_minutes = """
SELECT
  date_trunc('minute', created_at) AS minute,
  SUM(steps)    AS steps_sum,
  SUM(calories) AS calories_sum,
  AVG(steps)    AS steps_avg,
  AVG(calories) AS calories_avg
FROM activity
WHERE created_at >= now() - interval '60 minutes'
GROUP BY 1
ORDER BY 1;
"""

df_min = pd.read_sql(q_minutes, engine)

df_min["minute"] = pd.to_datetime(df_min["minute"])
df_min = df_min.sort_values("minute")

df_min.tail(10)

In [ ]:
plt.figure()
plt.plot(df_min["minute"], df_min["steps_sum"], label="steps_sum")
plt.plot(df_min["minute"], df_min["calories_sum"], label="calories_sum")
plt.title("Активность по минутам (последние 60 минут)")
plt.xlabel("minute")
plt.ylabel("value")
plt.legend()
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure()
plt.plot(df_min["minute"], df_min["steps_avg"], label="steps_avg")
plt.plot(df_min["minute"], df_min["calories_avg"], label="calories_avg")
plt.title("Средняя активность по минутам (последние 60 минут)")
plt.xlabel("minute")
plt.ylabel("avg")
plt.legend()
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()